# Imports

In [1]:
import sys
import os

sys.path.insert(0, "/".join(os.getcwd().split("/")[:-1]))
__package__ = "LoadModels_NoteBooks"


In [2]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from utils.global_functions import arg_parse, Metrics, MySampler, NewCrossEntropyLoss
from transformers import AutoModel
from  SingleModels.models.text import BertClassifier
from  SingleModels.text_nn import prepare_dataloader
import wandb
from utils.trainer import Trainer
from utils.data_loaders import BertDataset
import numpy as np

/home/zeerak.talat/.conda/envs/featuregen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
test = AutoModel.from_pretrained("roberta-large")

In [18]:
i = 0
for k in test.base_model.parameters():
    i += 1
print(i)
len(test.state_dict().keys())

391


391

# Load in Models/Path

In [3]:
args = {}
args["dropout"] =  0
args["output_dim"] =  7
args["dataset"] =  "meld"
args["BertModel"] =  "roberta-large"
args["hidden_size"] =  512
bert = BertClassifier(args)
path = "meld_iemo_text/de3djeoo/good-sweep-17/best.pt"

In [4]:
bert_state_dict = torch.load(f"../../TAV_Train/{path}" , map_location=torch.device('cpu'))['model_state_dict']
bert.load_state_dict(bert_state_dict)

<All keys matched successfully>

### Below is for encoder only load

In [22]:
bert_state_dict = torch.load(f"../../TAV_Train/{path}" , map_location=torch.device('cpu'))['model_state_dict']
roberta_state_dict = bert.state_dict()

for key in roberta_state_dict.keys():
    bert_key = 'bert.' + key  # prepend 'bert.' to the key
    if bert_key in bert_state_dict:
        # print(f"Loading {bert_key}"  , flush = True)
        roberta_state_dict[key] = bert_state_dict[bert_key]

# Check load properly

In [25]:
bert_state_dict['bert.encoder.layer.23.output.dense.bias']

tensor([-0.0754, -0.0241, -0.0955,  ...,  0.0763, -0.1577,  0.0778])

In [27]:
(roberta_state_dict['encoder.layer.23.output.dense.bias'] == bert_state_dict['bert.encoder.layer.23.output.dense.bias']).all()

tensor(True)

# Try Test Set

In [ ]:
bert.eval()

In [53]:
name = "../data/iemo.pkl"
df = pd.read_pickle(f"{name}")
number_index = "emotion"
label_index = "emotion_label"
label2id = (
    df.drop_duplicates(label_index).set_index(label_index).to_dict()[number_index]
)
id2label = {v: k for k, v in label2id.items()}
id2label

{0: 'frustrated', 1: 'neutral', 2: 'angry', 5: 'happy', 3: 'sad', 4: 'excited'}

In [56]:
df['emotion_label'].value_counts()

emotion_label
frustrated    1849
neutral       1708
angry         1103
sad           1084
excited       1041
happy          595
Name: count, dtype: int64

In [54]:
df_test = prepare_dataloader(
        df[df['split'] == "test"], name, 1, "emotion", 2 , feature_col = "text", check="test" , bert = "roberta-large"
    )
Metric = Metrics(num_classes=len(id2label), id2label=id2label, rank="cpu")

In [ ]:
trainer = Trainer(big_batch=23 , num_steps=1)
trainer.evaluate(bert, df_test, Metric)

In [58]:
from sklearn.metrics import f1_score
import numpy as np

# Your confusion matrix
cm = np.array([[1015,   98,   50,   53,   34,    2,    4],
        [  87,  254,   30,   23,    6,    0,    2],
        [  42,   37,  164,   29,    5,    2,    2],
        [  75,   48,   33,  162,   12,    9,    5],
        [  80,   22,    9,   26,   62,    1,    8],
        [  24,    8,    6,   16,    2,   11,    0],
        [  19,   11,    5,    4,    4,    0,    7]])

# Convert the confusion matrix into actual and predicted labels
y_true = np.repeat(np.arange(cm.shape[0]), np.sum(cm, axis=1))
y_pred = np.repeat(np.arange(cm.shape[1]), np.sum(cm, axis=0))

# Calculate weighted F1 score
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"The weighted F1 score is {f1}")


The weighted F1 score is 0.7320330343162528


# Try Loading in out model, but index out encoder

In [61]:
new_model = bert.bert
new_model.eval()
for param in new_model.base_model.parameters():
    print(param)
    

Parameter containing:
tensor([[-0.1404, -0.0095,  0.0388,  ...,  0.0507, -0.0061, -0.0360],
        [ 0.0078, -0.0156,  0.0156,  ..., -0.0156,  0.0230,  0.0156],
        [-0.0830, -0.0010, -0.1174,  ...,  0.1088,  0.0698, -0.0357],
        ...,
        [ 0.0393,  0.0031,  0.0465,  ..., -0.0240, -0.0505,  0.0342],
        [ 0.0498,  0.0272,  0.0412,  ..., -0.0370, -0.0100,  0.0071],
        [-0.0149, -0.0114, -0.0222,  ...,  0.0440,  0.0116, -0.0330]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0038,  0.0253, -0.0092,  ...,  0.0177,  0.0062, -0.0162],
        [ 0.0117, -0.0019, -0.0267,  ...,  0.0062, -0.0192,  0.0263],
        [ 0.0317,  0.0149, -0.0551,  ..., -0.0717, -0.0462,  0.0468],
        ...,
        [-0.0209, -0.0052,  0.0484,  ..., -0.0394,  0.0463,  0.0537],
        [-0.0274,  0.1172,  0.0470,  ...,  0.0169, -0.1204,  0.0525],
        [ 0.0969, -0.0729,  0.0558,  ..., -0.1204, -0.1074,  0.0488]],
       requires_grad=True)
Parameter containing:
tensor([[ 2.

In [20]:
meld_iemo = pd.read_pickle("../data/meld_iemo.pkl")

In [24]:
iemo = meld_iemo.tail(7380)
iemo.to_pickle("../data/iemo.pkl")

In [ ]:
old_df = pd.read_pickle(f"../iemo.pkl")
old_df['emotion_label'].value_counts()
old_df_labels = old_df[['emotion' , 'emotion_label']]
old_df_labels

In [52]:
iemo = pd.read_pickle("../data/iemo.pkl")
# iemo.reset_index(inplace = True, drop = True)
iemo

,talker,sentiment_label,dialog,split,text,sentiment,timings,bbox,audio_path,video_path,og_text,len,og_text_with_auth,emotion,emotion_label
0,MARY,frustrated,7,train,MARY: What's he going to say?,0,"(6.21, 9.32)","[[[65, 33, 200, 217]], [[63, 32, 203, 217]], [...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,What's he going to say?,16,MARY: What's he going to say?,0,frustrated
1,JAMES,neutral,7,train,MARY: What's he going to say? </s></s> JAMES: ...,1,"(11.9892, 15.1219)","[[[24, 37, 172, 217], [0, 86, 36, 219], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He saw it.,16,JAMES: He saw it.,1,neutral
2,JAMES,neutral,7,train,JAMES: He saw it. </s></s> JAMES: He was out h...,1,"(19.0456, 22.3962)","[[[19, 36, 175, 216], [2, 62, 54, 224], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He was out here when it broke.,16,JAMES: He was out here when it broke.,1,neutral
3,JAMES,neutral,7,train,JAMES: He was out here when it broke. </s></s>...,1,"(24.8, 35.9971)","[[[40, 37, 176, 217], [2, 74, 68, 220], [183, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,Four o clock this morning. I heard a crack a...,16,JAMES: Four o clock this morning. I heard a c...,1,neutral
4,JAMES,neutral,7,train,JAMES: Four o clock this morning. I heard a c...,1,"(38.66, 40.53)","[[[27, 35, 168, 218], [184, 129, 222, 205], [0...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,I don't know.,16,JAMES: I don't know.,1,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,WILLIAM,excited,149,val,"ELIZABETH: Oh, It burnt my comb and all the to...",4,"(215.4, 220.75)","[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"[LAUGHTER] That was quit a rouser, wasn't it?",16,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,excited
7376,WILLIAM,excited,149,val,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,"(221.13, 223.83)","[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"Oh, I didn't hit you very hard.",16,"WILLIAM: Oh, I didn't hit you very hard.",4,excited
7377,WILLIAM,excited,149,val,"WILLIAM: Oh, I didn't hit you very hard. </s><...",4,"(229.88, 233.06)","[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,I should never forget his face. [LAUGHTER],16,WILLIAM: I should never forget his face. [LAUG...,4,excited
7378,WILLIAM,excited,149,val,WILLIAM: I should never forget his face. [LAUG...,4,"(234.14, 236.66)","[[[62, 11, 190, 215]], [[60, 13, 189, 214]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,We were very much younger then.,16,WILLIAM: We were very much younger then.,4,excited


In [47]:
iemo.drop(columns=['emotion' , 'emotion_label'] , inplace = True)

In [50]:
new_iemo = pd.merge(iemo , old_df_labels , left_index=True , right_index=True )
new_iemo['emotion_label'].value_counts()

emotion_label
frustrated    1849
neutral       1708
angry         1103
sad           1084
excited       1041
happy          595
Name: count, dtype: int64

In [51]:
new_iemo.to_pickle("../data/iemo.pkl")